# Optimizing allowable vessel times
### 1 Traffic Generator
### 2 Terminal Design
### 3 Optimization
### 4 Sensitivity to market volatility

In [1]:
import copy
import plotly
import numpy as np
import pandas as pd

from plotly import tools
import plotly.tools as tls 
import plotly.plotly as py  
import plotly.graph_objs as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

import terminal_optimization.visualisation               as visualisation
import terminal_optimization.forecast                    as forecast
import terminal_optimization.investment_decisions        as invest
import terminal_optimization.infrastructure              as infra
import terminal_optimization.financial_analysis          as financial
import terminal_optimization.initial_terminal            as initial
import terminal_optimization.investment_decisions_future as future_invest

# Log in to Plotly servers
#plotly.tools.set_credentials_file(username='jorisneuman', api_key='zButeTrlr5xVETcyvazd')
#plotly.tools.set_credentials_file(username='wijzermans', api_key='FKGDvSah3z5WCNREBZEq')
#plotly.tools.set_credentials_file(username='wijnandijzermans', api_key='xeDEwwpCK3aLLR4TIrM9')
plotly.tools.set_credentials_file(username='evaijzermans', api_key='Aidg8GbABtpEm80ek2Zz')

# Applied colours within the plots
colors = ['rgba(55, 83, 109, 1)', 'rgba(55, 83, 109, 0.8)', 'rgba(55, 83, 109, 0.5)',
          'rgb(129, 180, 179, 1)', 'rgb(79, 127, 127, 1)', 'rgb(110,110,110)'] 

## 1 Traffic Generator

In [2]:
def traffic_generator(nr_generations, start_year, simulation_window):
    
    ###################################################################################################
    # Traffic projections on which estimate is based
    ###################################################################################################

    # Import commodities from package
    maize   = forecast.bulk_commodities(**forecast.maize_data)
    soybean = forecast.bulk_commodities(**forecast.maize_data)
    wheat   = forecast.bulk_commodities(**forecast.wheat_data)

    # Maize - demand
    maize.historic = [1100000,1200000,1250000,1320000,1400000]
    maize.rate     = 1.0586 
    maize.sigma    = 0.0374

    # If defined number of traffic generations is even, one iteration is added in order to be able to identify the median
    if nr_generations % 2 == 0:
        nr_generations = nr_generations + 1 

    # Create traffic projections
    traffic_projections = []
    for iterations in range(nr_generations):
        maize.random_scenario(start_year, simulation_window, maize.historic, maize.rate, maize.sigma)
        traffic_projections.append(maize.demand)
    
    # Calculate the median traffic projection (i.e. traffic scenario)
    traffic_scenario = []
    
    for i in range (len(maize.historic)):
        traffic_scenario.append(
            maize.historic[i])
        
    for year in range (len(maize.historic), len(maize.historic) + simulation_window):
        throughputs = []
        for i in range (len(traffic_projections)):
            throughputs.append(
                traffic_projections[i][year])
        traffic_scenario.append(
            np.median(throughputs))
        
    # Define commodity demand     
    maize.demand   = traffic_scenario 
    soybean.demand = [0]*len(maize.demand)
    wheat.demand   = [0]*len(maize.demand)
    commodities    = [maize, soybean, wheat]
    
    # Import vessels from package
    handysize = forecast.vessel(**forecast.handysize_data)
    handymax  = forecast.vessel(**forecast.handymax_data)
    panamax   = forecast.vessel(**forecast.panamax_data)
    vessels = [handysize, handymax, panamax]
    
    # Calculate yearly calls
    vessels = forecast.vessel_call_calc(vessels, commodities, simulation_window)

    # Plot forecast
    fig = visualisation.scenario(traffic_projections, commodities)
    
    return vessels, commodities, fig, traffic_projections, traffic_scenario

## 2 Terminal Design

In [3]:
def design(chosen_method, terminal, vessels, commodities, start_year, simulation_window):
    
    for i in range (start_year, start_year + simulation_window): 
        
        year = i 
        timestep = year - start_year

        ##################################################################
        # Investment Decisions (current performance method)
        ##################################################################     
        
        if chosen_method == 'Current performance method':
            
            # Berths and cranes
            terminal.berths, terminal.cranes = invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)

            # Quay
            terminal.quays = invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities,  year, timestep, operational_hours)

        ##################################################################
        # Investment Decisions (Perfect foresight and forecast method)
        ################################################################## 
        
        if chosen_method == 'Perfect foresight method' or chosen_method == 'Forecast based method':
            
            # Create forecast and accompanying vessel calcs
            commodities = forecast.forecaster(chosen_method, 'Linear', commodities, forecast_window, hindcast_window, timestep)
            vessels = forecast.forecast_call_calc(vessels, commodities, simulation_window)
            
            # Berths and cranes
            terminal.berths, terminal.cranes = future_invest.berth_invest_decision(terminal.berths, terminal.cranes, commodities, vessels, terminal.allowable_vessel_waiting_time, year, timestep, operational_hours)

            # Quay
            terminal.quays = future_invest.quay_invest_decision(terminal.quays, terminal.berths, vessels, year, timestep)

            # Storage
            storage_type = 'Silos'
            terminal.storage = future_invest.storage_invest_decision(terminal.storage, terminal.dwell_time, vessels, storage_type, commodities, year, timestep, operational_hours)

            # Loading stations
            terminal.stations, terminal.trains = future_invest.station_invest_decision(terminal.stations, forecast.train(**forecast.train_data), terminal.allowable_train_waiting_time, commodities, timestep, year, operational_hours)

            # Conveyors
            terminal.quay_conveyors = future_invest.quay_conveyor_invest_decision(terminal.quay_conveyors, terminal.berths, terminal.cranes, commodities, year, timestep, operational_hours)
            terminal.hinterland_conveyors = future_invest.hinterland_conveyor_invest_decision(terminal.hinterland_conveyors, terminal.stations, commodities, year, timestep, operational_hours)      
        
        ##################################################################
        # Business logic
        ################################################################## 
        
        # Terminal throughput
        terminal = financial.throughput_calc(terminal, commodities, vessels, terminal.trains, operational_hours, timestep, year)
        # Revenue
        terminal.revenues = financial.revenue_calc(terminal.revenues, terminal.throughputs, commodities, year, timestep)       
        # Capex
        terminal.capex = financial.capex_calc(terminal, year, timestep)
        # Labour costs
        terminal.labour = financial.labour_calc(terminal, year, timestep, operational_hours)
        # Maintenance costs
        terminal.maintenance = financial.maintenance_calc(terminal, year, timestep)
        # Energy costs
        terminal.energy = financial.energy_calc(terminal, year, operational_hours, timestep)
        # Insurance costs
        terminal.insurance = financial.insurance_calc(terminal, year, timestep)
        # Lease costs 
        terminal.lease = financial.lease_calc(terminal, year,timestep)
        # Demurrage costs
        terminal.demurrage = financial.demurrage_calc(terminal.demurrage, terminal.berths, terminal.cranes, commodities, vessels, operational_hours, timestep, year)
        # Residual value calculations 
        terminal.residuals = financial.residual_calc(terminal, year, timestep)
        # Profits
        terminal.profits = financial.profit_calc(terminal, simulation_window, timestep, year, start_year)
        # Opex
        terminal.opex = financial.opex_calc(terminal, year, timestep)  
        
    #WACC depreciated profits
    terminal.WACC_cashflows = financial.WACC_calc(terminal.project_WACC, terminal.profits, terminal.revenues, terminal.capex, terminal.opex, simulation_window, start_year)
    
    # Combine all cashflows
    terminal.cashflows = financial.cashflow_calc(terminal, simulation_window, start_year) 
    
    #NPV 
    terminal.NPV = financial.NPV_calc(terminal.WACC_cashflows)
        
    return terminal

## 3 Optimization

In [ ]:
# Simulation parameters
start_year        = 2018   # start year of simulation
simulation_window = 20     # forecast 20 years
end_year          = start_year + simulation_window - 1
operational_hours = 5840   # operational hours per year (16 hours per day 365 days a year)
project_WACC      = 0.095 # Applied project weighted average cost of capital
nr_generations    = 1001   # Number of traffic projections generated

# In case of future based methods:
chosen_method = 'Perfect foresight method'
hindcast_window = 5  # Number of years of materialized traffic volumes that are included in assessment
forecast_window = 2  # Number of years of forecasted traffic volumes

# Make traffic projections
vessels, commodities, fig, traffic_projections, traffic_scenario = traffic_generator(nr_generations, start_year, simulation_window)
commodities[0].utilization = 0.85
commodities[0].handling_fee = 9.80
maize = commodities[0]

# Define demurrage rates
vessels[0].demurrage_rate = 333     # Handysize rate
vessels[1].demurrage_rate = 375     # Handymax rate
vessels[2].demurrage_rate = 460     # Panamax rate

# Performance triggers (static values)
triggers = []
triggers.append([0.30, 'Acceptable waiting vessel time as factor of service time'])  # PIANC guidelines
triggers.append([18  , 'Dwell time of cargo in storage, presented in days'])         # PIANC guidelines
triggers.append([0.50, 'Acceptable waiting train time as factor of service time'])   # Reference project

# Design initial terminal
terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

# Spectrum of trigger values that will be evaluated
triggers_spectrum = [] 
triggers_spectrum.append([np.linspace(0.10, 1.51, 142), 'Acceptable waiting vessel time as factor of service time'])

# Iterate through allowable vessel waiting times
iterations = []
original_value = copy.deepcopy(triggers[0][0])
for waiting_time in triggers_spectrum[0][0]:
    triggers[0][0] = waiting_time
    terminal  = initial.terminal(project_WACC, triggers)
    terminal  = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)
    iteration = copy.deepcopy(terminal)
    iterations.append(iteration)
triggers[0][0] = original_value

plotdata = visualisation.NPV_distribution_vessel_waiting_times(iterations)
fig = dict(data=plotdata[0], layout=plotdata[1])
fig['layout'].update(
    title='The financial impact of allowable vessel waiting time',
    yaxis=dict(
        title='Project NPV',
        tickprefix='$',
        range=[5000000,50000000]),
        annotations=[dict(
                        x=1,
                        y=-0.20,
                        showarrow=False,
                        text='Source: URL or QR code',
                        xanchor='right',
                        xref='paper',
                        yref='paper'),
                    dict(
                        x=25,
                        y=39000000,
                        showarrow=False,
                        text='PIANC guidelines',
                        xanchor='center')],
        shapes=[dict(
                    type='line',
                    x0=20,
                    y0=0,
                    x1=20,
                    y1=37000000,
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=20,
                    y0=41500000,
                    x1=20,
                    y1=50000000,
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=30,
                    y0=0,
                    x1=30,
                    y1=37000000,
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
                dict(
                    type='line',
                    x0=30,
                    y0=41500000,
                    x1=30,
                    y1=50000000,
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot'))])

py.iplot(fig, filename='Trigger iteration - vessel waiting time')

## 4 Sensitivity to market volatility
- Scenario 1: Steady growth
- Scenario 2: Erratic growth
- Scenario 3: Crisis 

### Scenario 1 - Steady growth

In [15]:
demand = [1100000,1200000,1250000,1320000,1400000]
for i in range(20):
    demand.append(
        1400000+150000*i)
maize.demand = demand
scenarios.append(
    demand)

terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

# Performance triggers (iteration values)
triggers_spectrum = [] 
triggers_spectrum.append([np.linspace(0.10, 1.50, 141), 'Acceptable waiting vessel time as factor of service time'])

# Iterate through allowable vessel waiting times
iterations = []
original_value = copy.deepcopy(triggers[0][0])
for waiting_time in triggers_spectrum[0][0]:
    triggers[0][0] = waiting_time
    terminal  = initial.terminal(project_WACC, triggers)
    terminal  = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)
    iteration = copy.deepcopy(terminal)
    iterations.append(iteration)
triggers[0][0] = original_value

NPVs = []
for i in range(len(iterations)):
    NPVs.append(copy.deepcopy(iterations[i].NPV))
vessel_waiting_times.append(
    copy.deepcopy(NPVs))

### Scenario 2 - Erratic growth

In [16]:
demand = [1100000,1200000,1250000,1320000,1400000]
for i in range(20):
    if i == 0 or i == 3 or i == 6 or i == 9 or i == 12 or i == 15 or i == 18: 
        demand.append(
            demand[-1] - 140000)
    if i == 1 or i == 4 or i == 7 or i == 10 or i == 13 or i == 16 or i == 19: 
        demand.append(
            demand[-1] + 375000)
    if i == 2 or i == 5 or i == 8 or i == 11 or i == 14 or i == 17 or i == 20: 
        demand.append(
            demand[-1] + 200000)
maize.demand = demand
scenarios.append(
    demand)

terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

# Iterate through allowable vessel waiting times
iterations = []
original_value = copy.deepcopy(triggers[0][0])
for waiting_time in triggers_spectrum[0][0]:
    triggers[0][0] = waiting_time
    terminal  = initial.terminal(project_WACC, triggers)
    terminal  = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)
    iteration = copy.deepcopy(terminal)
    iterations.append(iteration)
triggers[0][0] = original_value

NPVs = []
for i in range(len(iterations)):
    NPVs.append(copy.deepcopy(iterations[i].NPV))
vessel_waiting_times.append(
    copy.deepcopy(NPVs))

### Scenario 3 - Crisis

In [17]:
demand = [1100000,1200000,1250000,1320000,1400000]
for i in range(20):
    if i < 5: 
        demand.append(
            demand[-1] + 150000)
    if i >= 5 and i < 8: 
        demand.append(
            demand[-1] - 265000)
    if i >= 8 and i < 13: 
        demand.append(
            demand[-1] + 490000)
    if i >= 13: 
        demand.append(
            demand[-1] + 150000)
maize.demand = demand
scenarios.append(
    demand)

terminal = initial.terminal(project_WACC, triggers)
terminal = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)

# Iterate through allowable vessel waiting times
iterations = []
original_value = copy.deepcopy(triggers[0][0])
for waiting_time in triggers_spectrum[0][0]:
    triggers[0][0] = waiting_time
    terminal  = initial.terminal(project_WACC, triggers)
    terminal  = design(chosen_method, terminal, vessels, commodities, start_year, simulation_window)
    iteration = copy.deepcopy(terminal)
    iterations.append(iteration)
triggers[0][0] = original_value

NPVs = []
for i in range(len(iterations)):
    NPVs.append(copy.deepcopy(iterations[i].NPV))
vessel_waiting_times.append(
    copy.deepcopy(NPVs))

## Combined visualization

In [21]:
trace1 = go.Scatter(
    name='Traffic volume',
    x=maize.years[5:],
    y=scenarios[0][5:],
    showlegend=False,
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=3,
    )
)

trace2 = go.Scatter(
    name='Traffic volume',
    x=maize.years[5:],
    y=scenarios[1][5:],
    showlegend=False,
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=3,
    )
)

trace3 = go.Scatter(
    name='Traffic volume',
    x=maize.years[5:],
    y=scenarios[2][5:],
    showlegend=False,
    mode='lines',
    line=dict(
        color='rgb(214, 39, 40)', 
        width=3,
    )
)

trace4 = {
    'x': triggers_spectrum[0][0]*100,
    'y': vessel_waiting_times[0],
    'name': 'Commodity dwell time',
    'type': 'bar',
    'showlegend': False,
    'marker': dict(color=colors[0])
}

trace5 = {
    'x': triggers_spectrum[0][0]*100,
    'y': vessel_waiting_times[1],
    'name': 'Commodity dwell time',
    'type': 'bar',
    'showlegend': False,
    'marker': dict(color=colors[0])
}

trace6 = {
    'x': triggers_spectrum[0][0]*100,
    'y': vessel_waiting_times[2],
    'name': 'Commodity dwell time',
    'type': 'bar',
    'showlegend': False,
    'marker': dict(color=colors[0])
}

fig = tools.make_subplots(rows=3,
                          cols=7,
                          specs=[[{'colspan': 2}, None, None, {'colspan': 4}, None, None, None],
                                 [{'colspan': 2}, None, None, {'colspan': 4}, None, None, None],
                                 [{'colspan': 2}, None, None, {'colspan': 4}, None, None, None]])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 3, 1)
fig.append_trace(trace4, 1, 4)
fig.append_trace(trace5, 2, 4)
fig.append_trace(trace6, 3, 4)

fig['layout'].update(
    height=1350, 
    width=900, 
    title='Trigger optimization under various economic scenarios',
    xaxis = dict(
        range=[2018,2037]),
    xaxis3 = dict(
        range=[2018,2037]),
    xaxis5 = dict(
        range=[2018,2037]),
    xaxis2 = dict(
        title='Allowable vessel waiting time',
        range=[9,141],
        ticksuffix='%',
        tickangle=45,
        dtick = 10),
    xaxis4 = dict(
        title='Allowable vessel waiting time',
        range=[9,141],
        ticksuffix='%',
        tickangle=45,
        dtick = 10),
    xaxis6 = dict(
        title='Allowable vessel waiting time',
        range=[9,141],
        ticksuffix='%',
        tickangle=45,
        dtick = 10),
    
    yaxis = dict(
        range=[10000,4050000],
        title='Traffic volume (t/year)',
        ticksuffix = 't',
        dtick=1000000),
    yaxis3 = dict(
        range=[10000,4050000],
        title='Traffic volume (t/year)',
        ticksuffix = 't',
        dtick=1000000),
    yaxis5 = dict(
        range=[10000,4050000],
        title='Traffic volume (t/year)',
        ticksuffix = 't',
        dtick=1000000),
    yaxis2 = dict(
        range=[-11000000,12500000],
        title='Project NPV',
        tickprefix = '$',
        dtick=10000000),
    yaxis4 = dict(
        range=[-11000000,12500000],
        title='Project NPV',
        tickprefix = '$',
        dtick=10000000),
    yaxis6 = dict(
        range=[-11000000,12500000],
        title='Project NPV',
        tickprefix = '$',
        dtick=10000000),
    margin=dict(
            l=90,
            r=50,
            b=150,
            t=70),
    annotations=[dict(
                    x=1,
                    y=-0.10,
                    showarrow=False,
                    text='Source: URL or QR code',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=0.49,
                    y=0.785,
                    showarrow=False,
                    text='PIANC guidelines',
                    xanchor='left',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=0.49,
                    y=0.405,
                    showarrow=False,
                    text='PIANC guidelines',
                    xanchor='left',
                    xref='paper',
                    yref='paper'),
                dict(
                    x=0.49,
                    y=0.025,
                    showarrow=False,
                    text='PIANC guidelines',
                    xanchor='left',
                    xref='paper',
                    yref='paper')],
    
    shapes=[dict(   type='line',
                    x0=0.493,
                    y0=0,
                    x1=0.493,
                    y1=0.02,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
            dict(   type='line',
                    x0=0.493,
                    y0=0.05,
                    x1=0.493,
                    y1=0.26,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.493,
                    y0=0.37,
                    x1=0.493,
                    y1=0.39,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.493,
                    y0=0.42,
                    x1=0.493,
                    y1=0.64,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
            dict(   type='line',
                    x0=0.493,
                    y0=0.74,
                    x1=0.493,
                    y1=0.76,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.493,
                    y0=0.79,
                    x1=0.493,
                    y1=1,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
            dict(   type='line',
                    x0=0.531,
                    y0=0,
                    x1=0.531,
                    y1=0.02,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
            dict(   type='line',
                    x0=0.531,
                    y0=0.05,
                    x1=0.531,
                    y1=0.26,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.531,
                    y0=0.37,
                    x1=0.531,
                    y1=0.39,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.531,
                    y0=0.42,
                    x1=0.531,
                    y1=0.64,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
            dict(   type='line',
                    x0=0.531,
                    y0=0.74,
                    x1=0.531,
                    y1=0.76,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot')),
           dict(   type='line',
                    x0=0.531,
                    y0=0.79,
                    x1=0.531,
                    y1=1,
                    xref='paper',
                    yref='paper',
                    line=dict(
                        color='rgb(170, 170, 170, 0.8)',
                        width=2,
                        dash='dashdot'))])

py.iplot(fig, filename='stacked-subplots')

This is the format of your plot grid:
[ (1,1) x1,y1           -      ]      (empty)      [ (1,4) x2,y2           -                -                -      ]
[ (2,1) x3,y3           -      ]      (empty)      [ (2,4) x4,y4           -                -                -      ]
[ (3,1) x5,y5           -      ]      (empty)      [ (3,4) x6,y6           -                -                -      ]

